<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/24_06_2024_Finding_the_Origination%2C_Destination%2C_and_stops_in_a_give_lineID_for_given_direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Finding the Origination, Destination, and stops in a give lineID for given direction

In [1]:
import requests
import csv
import re
import time

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {
        'serviceTypes': 'Regular'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates[0]:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Extract numeric part from stop ID using regex
                    numeric_stop_id = re.search(r'\d+', stop_id).group() if stop_id else None

                    # Append to all_stop_details only if numeric part of stop ID has not been printed before
                    if numeric_stop_id and numeric_stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': numeric_stop_id,
                            'Stop Name': stop_name
                        })

                        # Add numeric stop ID to printed set
                        printed_stop_ids.add(numeric_stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Example usage
if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")

    fetch_route_details(line_id, direction)

Enter line ID: 214
Enter direction (e.g., inbound or outbound): inbound
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---
Data written to 'stop_details.csv' successfully.
Data from CSV:
1      490015209  Finsbury Square / Moorgate
2      490006850  Finsbury Square / Moorgate
3      490003831  Finsbury Street
4      490000149  Moorgate Station
5      490006612  Epworth Street
6      490000169  Old Street Station
7      490015195  Old Street
8      490015194  Old Street Station
9      490010109  Old Street Roundabout
10     490015193  Old Street Station
11     490010108  Old Street Station
12     490011249  Moorfields Eye Hospital
13     490010012  Moorfields Eye Hospital
14     490018497  Bath Street
15     490014780  Windsor Terrace
16     490016371  Shoreditch Police Station
17     490018499  Ironmonger Row  Norman Street
18     490018498  Radnor Street  Lizard Street
19     490004937  Central Stre

In [2]:
import requests
import csv
import re
import time

def fetch_route_details(line_id, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route"
    params = {
        'serviceTypes': 'Regular'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        route_sections = data.get('routeSections', [])

        found_direction = False

        for section in route_sections:
            if section.get('direction', '').lower() == direction.lower():
                found_direction = True
                origination_name = section.get('originationName', 'Unknown')
                destination_name = section.get('destinationName', 'Unknown')
                originator = section.get('originator', 'Unknown')
                destination = section.get('destination', 'Unknown')

                print(f"Direction: {direction}")
                print(f"Origination Name: {origination_name}")
                print(f"Destination Name: {destination_name}")
                print(f"Originator: {originator}")
                print(f"Destination: {destination}")
                print("---")

                # Fetch and print sequence of stop points
                all_stop_details = fetch_and_append_stop_points(line_id, originator, destination, direction)

                if all_stop_details:
                    # Write to CSV file
                    write_to_csv(all_stop_details)

                    # Display data from CSV file
                    display_data_from_csv()

        if not found_direction:
            print(f"No route found for direction '{direction}'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def fetch_and_append_stop_points(line_id, originator, destination, direction):
    url = f"https://api.tfl.gov.uk/Line/{line_id}/Route/Sequence/{direction}"
    all_stop_details = []

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes

        data = response.json()
        line_strings = data.get('lineStrings', [])

        if line_strings:
            printed_stop_ids = set()  # To track printed stop IDs

            for line_string in line_strings:
                # Parsing the line string to extract latitudes and longitudes
                coordinates = eval(line_string)  # Evaluating string to list
                for lon, lat in coordinates[0]:
                    # Fetching and appending stop details using the lat and lon
                    fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details)

        else:
            print("No stop points found for this route.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stop points: {e}")

    return all_stop_details

def fetch_and_append_stop_details(lat, lon, printed_stop_ids, all_stop_details, retries=3):
    url = f"https://api.tfl.gov.uk/Stoppoint?lat={lat}&lon={lon}&stopTypes=NaptanPublicBusCoachTram&radius=200"

    try:
        for attempt in range(retries):
            response = requests.get(url)

            if response.status_code == 429:
                time.sleep(2**attempt)  # Exponential backoff
                continue

            response.raise_for_status()  # Raise exception for bad status codes

            data = response.json()
            stop_points = data.get('stopPoints', [])

            if stop_points:
                for stop_point in stop_points:
                    stop_id = stop_point.get('id', 'Unknown')

                    # Append to all_stop_details only if stop ID has not been printed before
                    if stop_id and stop_id not in printed_stop_ids:
                        stop_name = stop_point.get('commonName', 'Unknown')

                        all_stop_details.append({
                            'Sl. No.': len(all_stop_details) + 1,
                            'Stop ID': stop_id,
                            'Stop Name': stop_name
                        })

                        # Add stop ID to printed set
                        printed_stop_ids.add(stop_id)

            else:
                # If no stop points found, continue to next attempt
                continue

            break  # Exit loop if successful

        else:
            # If maximum retries exceeded, handle quietly (you can add logging or alternative handling here)
            pass

    except requests.exceptions.RequestException as e:
        # Handle other request exceptions (you can add logging or alternative handling here)
        pass

def write_to_csv(all_stop_details):
    headers = ['Sl. No.', 'Stop ID', 'Stop Name']

    with open('stop_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        for stop_detail in all_stop_details:
            writer.writerow(stop_detail)

    print(f"Data written to 'stop_details.csv' successfully.")

def display_data_from_csv():
    print("Data from CSV:")

    with open('stop_details.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        # Print headers
        headers_printed = False

        for row in reader:
            if not headers_printed:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")
                headers_printed = True
            else:
                print(f"{row['Sl. No.']:<6} {row['Stop ID']:<10} {row['Stop Name']}")

# Example usage
if __name__ == "__main__":
    line_id = input("Enter line ID: ")
    direction = input("Enter direction (e.g., inbound or outbound): ")

    fetch_route_details(line_id, direction)


Enter line ID: 214
Enter direction (e.g., inbound or outbound): inbound
Direction: inbound
Origination Name: Finsbury Square
Destination Name: Highgate School
Originator: 490015209C
Destination: 490008160N
---
Data written to 'stop_details.csv' successfully.
Data from CSV:
1      490015209K Finsbury Square / Moorgate
2      490015209C Finsbury Square
3      490006850J Finsbury Square / Moorgate
4      490006850G Finsbury Square
5      490006850H Finsbury Square
6      490003831E Finsbury Street
7      490015209D Finsbury Square
8      490003831W Finsbury Street
9      490000149B Moorgate Station
10     490000149M Moorgate
11     490006850F Finsbury Square
12     490006612E Epworth Street
13     490000169ZA Old Street Station
14     490015195N Old Street
15     490015195M City Road / Leonard Street
16     490000169K Shoreditch Fire Station
17     490015194F Old Street Station
18     490000169L Old Street Station
19     490010109E Old Street Roundabout
20     490015194G Bunhill Row
21   